# Gym

## Link utili:

* Sito OpenAI: https://openai.com/

* GitHub OpenAI: https://github.com/openai

* Url Gym: https://gym.openai.com/

* GitHub Gym: https://github.com/openai/gym

* Wiki di Gym: https://github.com/openai/gym/wiki


## Giochiamo a Space Invaders!

![img](img/space_invaders.png)

In [ ]:
import gym
import time
import numpy as np

Creiamo una funzione che **genera episodi** dell'ambiente passato come parametro.

In [ ]:
def generate_episodes(env, sleep_seconds=0, get_action=None, max_steps=float('inf'), num_episodes=1,
                   render=True, verbosity=0):
    '''
    Genera episodi di un ambiente gym
    
    Parametri
    ---------
    env : un ambiente gym
    sleep_seconds : tempo di attesa in secondi tra due frame consecutivi
    get_action : una funzione che prende un'osservazione e restituisce un'azione.
                 Se None le azioni sono scelte casualmente
    max_steps : numero massimo di step dell'episodio. La funziona ritorna quando
                l'episodio è terminato o quando si è raggiunmto il numero di step
                massimo.
    num_episodes : numero di episodi da generare
    render : se True visualizza gli stati dell'ambiente
    verbosity : 0, 1, 2. Quante informazioni stampare a video
    
    Ritorno
    -------
    Una lista con le lunghezze degli episodi generati
    '''
    # lista contenente le lunghezze degli episodi
    <IMPLEMENTA LA FUNZIONE>

Facciamo una partita a **Space Invaders**

In [ ]:
env = gym.make('SpaceInvaders-v0')
generate_episodes(env, sleep_seconds=0.01, verbosity=1)

Alcune osservazioni:
* Nei giochi **Atari** `info` (l'ultimo elemento della tupla ritornata da `step`) comunica quante **vite** sono rimaste.
<br><br>
* observation è un **array numpy** di dimensioni (210, 160, 3) che rappresenta un frame di gioco. Le prime due dimensioni rappresentano rispettivamente la largezza e l'altezza del frame, la terza i canali RGB.
<br><br>
* I **missili** sparati dalle astronavi **non sono visibili** in tutti i frame a causa delle **limitazioni hardware** dell'Atari 2600. A riguardo è possibile leggere [questo interssante articolo](https://www.wired.com/2009/03/racing-the-beam).

## La classe Env
La classe più importante nell'architettura di Gym è `gym.Env`. È una classe astratta da cui ereditano tutti gli ambienti definiti nel package `gym.envs`.
<br>
I metodi principali della classe `Env` sono:
* `step`: esegue un **passo** della simulazione prendendo in input un'azione. Ritorna la tupla (observation, reward, done, info) in cui **observation** è lo stato dell'ambiente dopo l'esecuzione dell'azione, **reward** è la ricompensa ricevuta, **done** vale True se l'episodio è terminato, **info** contiene informazioni ausiliarie.
* `reset`: **inizializza** l'ambiente ritornandone il primo stato. Va chiamata **all'inizio** di ogni nuovo episodio.
* `render`: ritorna una **rappresentazione grafica** dello stato corrente dell'ambiente. Ha le seguenti opzioni:
 * `human`: visualizza lo stato all'interno di una finestra.
 * `rgb_array`: ritorna un **array numpy** di dimensioni (larhezza, altezza, 3) che rappresenta un'immagine dello stato corrente dell'ambiente.
 * `ansi`: ritorna una **stringa** che rappresenta una visualizzazione adatta ad un **terminale** dello stato corrente.
* `close`: esegue operazioni di **pulizia** finale.
* `seed`: imposta il seme del generatore di **numeri causali** dell'ambiente.

Il modo più semplice per **creare** un ambiente è utilizzare la funzione `gym.make` passandogli l'id dell'ambiente.
<br>
Si può creare un **nuovo ambiente** creando una classe che **eredita** da `Env` e ridefinendo i suoi metodi.

## La classe Space

Ogni ambiente ha un `action_space` (con le possibili azioni) e un `observation_space` (con i possibili stati) come attributi.
Tutti gli **spazi** sono sottoclassi di gym.Space e sono definiti nel package `gym.spaces`.

I possibili spazi sono:
* `Discrete(n)`: i suoi elementi sono i numeri interi da $0$ a $n-1$.
* `Box`: rappresenta uno spazio continuo. Ad esempio lo spazio degli stati dei giochi Atari è di tipo Box(210, 160, 3).
* `Multibinary`: tupla di valori binari (0,1).
* `Multidiscrete`: Tupla di variabili discrete.
* `Tuple`: prodotto cartesiano di spazi.

I metodi principali della classe `Space` sono
* `contains`: restituisce `True` se gli viene passato un **elemento dello spazio** come parametro.
* `sample`: ritorna un elemento **casuale** dello spazio.

Analizziamo lo spazio delle azioni e lo spazio degli stati di **Space Invaders**.

In [ ]:
from gym.envs.atari.atari_env import ACTION_MEANING
env = gym.make('SpaceInvaders-v0')
print(f"action space = {env.action_space}")
print(f"observation space = {env.observation_space}")
print(f"observation space shape = {env.observation_space.shape}")
# azioni possibili e loro descrizione
print({k : v for k, v in ACTION_MEANING.items() if k in env.action_space})
# gli spazi continui (di tipo Box) ammettono un massimo e un minimo per ogni elemento
print(f"observation space lower limits = {env.observation_space.low}")
print(f"observation space upper limits = {env.observation_space.high}")

### Discrete
È composto dai **numeri interi** da $0$ a $n$.

In [ ]:
# lo spazio dei numeri interi da 0 a 9
discrete = gym.spaces.Discrete(10)
print(f"spazio = {discrete}")
print(f"n = {discrete.n}")
print(f"sample = {discrete.sample()}")
print(f"contiene 10 è {discrete.contains(10)}")
print(f"contiene 9 è {discrete.contains(9)}")

### Box
È uno spazio **continuo**.

In [ ]:
# spazio degli array numpy 2 x 3 in cui per ogni elemento vengono
# specificati i valori minimo e massimo
low = np.array([[-1.,-2.,-3.],[-2.,0.,-4.]])
hi = np.array([[3.,4.,7.],[1.,10.,4.]])
box = gym.spaces.Box(low, hi)
print(f"spazio = {box}")
print(f"sample = {box.sample()}")

### Multibinary

I suoi elementi sono tuple di valori **binari**.

In [ ]:
# spazio delle tuple binarie composte da 5 elementi
multi_binary = gym.spaces.MultiBinary(5)
multi_binary.sample()

### Multidiscrete

* I suoi elementi sono **tuple di interi**

In [ ]:
# terne di interi in cui il primo elemento va da 0 a 6, iil secondo da 0 a 8, il terzo da 0 a 2
multi_discrete = gym.spaces.MultiDiscrete([7, 9, 3])
multi_discrete.sample()

### Tupla

E' il **prodotto cartesiano** di spazi

In [ ]:
# coppie in cui il primo elemento è di tipo MultiBinary e il secondo di tipo MultiDiscrete
tuple_ = gym.spaces.Tuple((gym.spaces.MultiBinary(5), gym.spaces.MultiDiscrete([7, 9, 3])))
print(f"spazio = {tuple_}")
tuple_.sample()

## Ambienti disponibili

Sono registrati nel file `__init__.py` del package `gym/envs`.

In [ ]:
envs = gym.envs.registry.env_specs
print(envs.keys())

### Algorithmic

L'agente deve programmare delle **macchine di Turing** ad eseguire semplici compiti, come **copiare una stringa**. Ogni macchina ha un nastro di input e uno di output, che possono essere uni o bi-dimensionali. Ad ogni istante l'agente deve deve:
* **spostare la testina** del nastro di input in una delle due (o quattro nel caso bidimensionale) direzioni possibili
* decidere se vuole **scrivere o meno** sul nastro di **output**
* in caso positivo, **selezionare il carattere**

In [ ]:
generate_episodes(gym.make('Copy-v0'), sleep_seconds=0.1)

## Atari

Un **emulatore** dell' [Atari 2600](https://it.wikipedia.org/wiki/Atari_2600) con diversi giochi.

In [ ]:
generate_episodes(gym.make('SpaceInvaders-v0'), sleep_seconds=0.1)

## Box2D

Ambienti che utilizzano la libreria di **fisica bidimensionale** [Box2D](https://box2d.org/).

In [ ]:
generate_episodes(gym.make('LunarLander-v2'), sleep_seconds=0.1)

## Classic control

**Sistemi fisici** estremamente **semplici** come il pendolo, il pendolo doppio e il pendolo inverso (**Cart-Pole**)

In [ ]:
generate_episodes(gym.make('CartPole-v0'), sleep_seconds=0.1)

## Toy text

Semplici ambienti con visualizzazione **testuale**.
<br>
Nel **Frozen Lake** l'agente deve raggiungere la casella G (Goal) evitando le caselle H (Hole).

In [ ]:
generate_episodes(gym.make('FrozenLake8x8-v0'), sleep_seconds=0.1)

## I wrappers

Un ambiente può essere creato invocando direttamente il **costruttore** della sua classe, oltre che tramite la funzione `gym.make`.

In [ ]:
from gym.envs.box2d.bipedal_walker import BipedalWalker
env = BipedalWalker()
generate_episodes(env, sleep_seconds=0.1, max_steps=50)

Il comportamento di un ambiente può essere modificato, o nuove funzionalità possono esseregli aggiunte, utilizzando un oggetto **wrapper** (`gym.core.Wrapper`).
<br><br>
Possiamo monitorare un addestramento utilizzando il wrapper `Monitor` che salva su disco le **statistiche** e i **video** degli episodi.

In [ ]:
from gym.wrappers.monitor import Monitor

# salva i file nella sottocartella tmp della cartella corrente
env = Monitor(gym.make("CartPole-v0"), directory="./tmp", force=True)
generate_episodes(env, sleep_seconds=0, num_episodes=5, verbosity=0)

Possiamo impostare un numero di step dopo i quali **interrompere l'episodio** anche se non è terminato, utilizzando `TimeLimit`.

In [ ]:
from gym.wrappers.time_limit import TimeLimit

env = TimeLimit(BipedalWalker(), max_episode_steps=10)
generate_episodes(env, sleep_seconds=0.1, verbosity=0)

Possiamo limitare il **range dei reward** ad un determinato intervallo tramite `ClipReward`.

In [ ]:
from gym.wrappers.clip_reward import ClipReward

# limita a 0.5 il massimo reward
rewards = []
env = ClipReward(gym.make("CartPole-v0"), min_r=0, max_r=0.5)
env.reset()
done = False
while not done:
    env.render()
    observation, reward, done, info = env.step(env.action_space.sample())
    rewards.append(reward)
    time.sleep(0.1)
env.close()
# nel Cart Pole l'agente ottiene un reward di 1 ad ogni passo.
# Dopo il clipping i reward valgono tutti 0.5
print(rewards)

Ci sono altri wrapper oltre a quelli appena visti (nel package `gym.wrappers`) e naturalmente è possibile crearne di nuovi.
<br>
È inoltre possibile **comporre** più wrapper.
<br>
Possiamo ad esempio imporre **contemporaneamente** un limite alla **lunghezza** degli episodi e al **range** dei reward.

In [ ]:
env = TimeLimit(ClipReward(BipedalWalker(), min_r=0, max_r=0.5), max_episode_steps=10)

Ogni wrapper espone l'ambiente (o il wrapper) **precedente** con l'attributo `env` e l'ambiente **originario** con l'attributo `unwrapped`.

In [ ]:
print(env)
print(env.env)
print(env.unwrapped)
# come env.unwrapped
print(env.env.env)

## Gli ambienti registrati

Possiamo vedere quali sono gli ambienti registrati consultando il **registro globale**.

In [ ]:
sorted([spec for spec in gym.envs.registry.env_specs])

Un ambiente registrato viene creato tramite la funzione `gym.make`.

In [ ]:
cartpole_v0 = gym.make("CartPole-v0")

Gli ambienti registrati sono gli ambienti **ufficiali** su cui possiamo testare i nostri algortimi e **confrontarli** con quelli degli altri.  [Questa pagina](https://github.com/openai/gym/wiki/Leaderboard) ospita la **classifica** dei migliori algoritmi (in termini di numero di episodi necessari a superare la sfida). Studiarli è un otimo modo per accrescere le proprie competenze di Reinforcement Learning.
<br><br>
Ad ogni ambiente registrato corrispondono delle **specifiche** (un ogetto della classe `EnvSpec`) accessibili tramite l'attrivuto `spec`.
<br><br>
Il **CartPole** è presente con le versioni 0 e 1. Nella prima ogni episodio ha una durata massimo di 200 passi (ottenuta con il wrapper TimeLimit). Il compito è considerato **risolto** quando gli ultimi 100 episodi hanno una durata media di almeno 195 passi.

In [ ]:
print("soglia = {}, massimo numero di passi per episodio = {}".format(cartpole_v0.spec.reward_threshold, 
                                                                      cartpole_v0.spec.max_episode_steps))

## Un algoritmo per il pendolo inverso

In questa sezione ti proponiamo una piccola **sfida**: ideare un algoritmo che risolva il problema del **Cart Pole**.
<br>
Il Cart Pole è un semplice sistema fisico costituito da un'**asta** imperniata ad un **carrello**, libera di ruotare. È completamente descritto da **quattro parametri**: la posizione del carrello, la sua velocità, l'angolo dell'asta, la sua velocità angolare. L'obiettivo è di tenere **l'asta in equilibrio** senza allontanare troopo il carrello dalla posizione centrale.

Un'occhiata alla **documentazione** della classe `CartPoleEnv` potrebbe esserti utile.

In [ ]:
print(gym.envs.classic_control.CartPoleEnv.__doc__)

La funzione `get_action` prende in input uno **stato** e restituisce un'**azione** (per ora scelta a caso). Ti chiediamo di implementarla in modo tale da raggiungere l'obiettivo di una durata media di almeno **195 passi** sui 200 massimi, clacolata su 100 episodi.

In [ ]:
def get_action(observation):
    <IMPLEMENTA LA FUNZIONE>

In [ ]:
env = gym.make("CartPole-v0")
lenghts = generate_episodes(env, get_action=get_action, num_episodes=100, render=False)

if np.mean(lenghts) >= 195:
    print("Bravo, hai superato la prova!")
else:
    print(f"La lunghezza media dei 100 episodi è stata di {np.mean(lenghts)}. Devi migliorare!")